In [138]:
import pandas as pd
df = pd.read_csv('src/sf/campaign_finance_proof_of_concept_460a.csv')
df = df[['canonical_filer_naml','canonical_tran_name','tran_amt2']]


In [139]:
from collections import defaultdict
class Vertex:
    def __init__(self, node):
        self.id = node
        self.adjacent = defaultdict(int)

    def __str__(self):
        return str(self.id) + ' adjacent: ' + str([x.id for x in self.adjacent])

    def add_neighbor(self, neighbor, weight=0):
        self.adjacent[neighbor] += weight

    def get_connections(self):
        return self.adjacent.keys()  

    def get_id(self):
        return self.id

    def get_weight(self, neighbor):
        return self.adjacent.get(neighbor)

class Graph:
    def __init__(self):
        self.vert_dict = {}
        self.num_vertices = 0

    def __iter__(self):
        return iter(self.vert_dict.values())

    def add_vertex(self, node):
        self.num_vertices = self.num_vertices + 1
        new_vertex = Vertex(node)
        self.vert_dict[node] = new_vertex
        return new_vertex

    def get_vertex(self, n):
        if n in self.vert_dict:
            return self.vert_dict[n]
        else:
            return None

    def add_edge(self, frm, to, amount = 0):
        frm_vertex = self.get_vertex(frm)
        if frm_vertex is None:
            frm_vertex = self.add_vertex(frm)
        
        to_vertex = self.get_vertex(to)
        if to_vertex is None:
            to_vertex = self.add_vertex(to)

        
        frm_vertex.add_neighbor(to_vertex, amount)
        # graph is directed -- leave commented out
        #self.vert_dict[to].add_neighbor(self.vert_dict[frm], amount)

    def get_vertices(self):
        return self.vert_dict.keys()

In [142]:
import csv
import decimal 
from collections import defaultdict


dgraph = Graph()

fname = 'src/sf/campaign_finance_proof_of_concept_460a.csv'
for row in csv.DictReader(open(fname, 'r'), delimiter=',', quotechar='"'):
    doner = row['canonical_tran_name']
    donee = row['canonical_filer_naml']
    amount = decimal.Decimal(row['tran_amt2'].replace("$","").replace(",",""))
    dgraph.add_edge(doner, donee, amount = amount)
    print(f'{doner} | {donee} | {amount}')

Arnold, John | Action Now Initiative | 30000.00
Arnold, John | Action Now Initiative | 130000.00
Arnold, John | Action Now Initiative | 880000.00
Arnold, John | Action Now Initiative | 1630000.00
Arnold, Laura | Action Now Initiative | 30000.00
Arnold, Laura | Action Now Initiative | 130000.00
Arnold, Laura | Action Now Initiative | 880000.00
Arnold, Laura | Action Now Initiative | 1630000.00
Bloomberg, Michael | Yes on Proposition V | 150000.00
Bloomberg, Michael | Yes on Proposition V | 1945497.50
Action Now Initiative | Yes on Proposition V | 10000.00
Action Now Initiative | Yes on Proposition V | 60000.00
Action Now Initiative | Yes on Proposition V | 185000.00
Action Now Initiative | Yes on Proposition V | 1185000.00
Action Now Initiative | Yes on Proposition V | 2185000.00


In [144]:
node_name = 'Arnold, John'
node_name = 'Arnold, Laura'
vertex = dgraph.get_vertex(node_name)

def search(vertex, graph, path=[]):
    if len(vertex.get_connections()) == 0:
        return path
    else:
        for connection in vertex.get_connections():
            path.append((connection.id, vertex.get_weight(connection)))
            return search(graph.get_vertex(connection.id), graph, path)
        

print(search(vertex, dgraph))

[('Action Now Initiative', Decimal('2670000.00')), ('Yes on Proposition V', Decimal('3625000.00'))]
